In [ ]:
# Configuration
# This section serves to configure the Starter Kit by importing 
# the necessary Python libraries that will be used in the rest of this notebook.

import support as sp
import visualization as vis

import pkg_resources

import matplotlib
%matplotlib inline


# In order to make the results in this notebook reproducible, this notebook was last tested using Python 3.9 and 
# the following versions of the imported modules: matplotlib, pandas, numpy, seaborn and geopy.

def try_package(x):
    pkg, pkg_version = x.split('==')
    try:
        version = pkg_resources.get_distribution(pkg).version
        if pkg_version == version:
            pass
        else:
            print('Warning: this notebook was tested with version %s of %s, but you have %s installed' %
                  (pkg_version, pkg, version))
    except Exception as e:
        print(e)
for p in ["matplotlib==3.6.0", "pandas==1.5.0","numpy==1.23.3", "seaborn==0.12.0", "geopy==2.2.0"]:
    try_package(p)

import warnings
warnings.filterwarnings('ignore')

# Starter Kit 3.3: Feature Engineering

## Description

Most data mining and machine learning algorithms do not work well if you just feed them your raw data: such data often contains noise and the most relevant distinguishing information is often implicit. For example, raw sensor data just contains a timestamp and a corresponding value, while interesting aspects of it might be the trends contained within or the number of times a treshold is exceeded. 

Feature engineering is the process of extracting from the raw data the most relevant distinguishing characteristics that will be presented to the algorithm during modeling. It is a way of deriving new information from the existing data and the interaction between its variables, so that this is explicitly represented and can be used by the algorithm. 

Feature engineering is one of the most important and most creative steps in the data science workflow. However, there is no clearly-defined formal process for engineering features and, consequently, this requires a lot of creativity, a good understanding of the domain and of the available data, some trial-and-error, etc. It is also desirable to have upfront some ideas of the modeling and analysis approach that may solve the problem at hand.
 
## Business goal

The overall goal of this Starter Kit is to present some **advanced feature engineering steps** related to **feature construction** and **extraction**. By keeping in mind **what is your business question** and **what is the corresponding data science task**, you will be able to derive **valuable features** that can be used in the next stage of your analysis.

## Application context

Feature engineering is one of the steps in the data science workflow with the most decisive impact on the accuracy of the model you want to develop. It is the final step before actually training the model and it defines how the input data will be presented to the model. 

## Starter Kit outline

In this Starter Kit we use a dataset from the Regional Rail System of Pennsylvania. Before starting the feature extraction _per se_, we will first apply some basic preprocessing to the dataset and have a high-level overview of the data. We will then derive several interesting features from the dataset that can be used to answer a question such as "can we predict delays at a given train station?"

> TWT: Below what was in the "Data requirements" section. This does not make sense (it literally says these requirements are not necessary ... then how is it a requirement?) + you don't necessarily need a multivaraite dataset for feature engineering.  

> You will need a multivariate dataset. One of the variables in the dataset will be used as the target variable in the next modeling stage, but for this Starter Kit it does not need to be defined.

>PDAG: do we assume that the reader know what a target variable is? + if it's not needed why do we keep it? for uniformity amongst SK?

## Data understanding

In order to illustrate how to engineer features, we will use in this Starter Kit a dataset provided by the Southeastern Pennsylvania Transportation Authority (SEPTA), which can be downloaded [here](https://www.kaggle.com/septa/on-time-performance). 

The SEPTA Regional Rail System consists of 13 branches and more than 150 stations in Philadelphia, Pennsylvania, and its suburbs and satellite cities, as depicted in the map below.

<img src="septaMap.png">

SEPTA uses On-Time Performance (OTP) to measure service reliability. OTP identifies the number of trains for all rail lines that arrive at their scheduled destination at the scheduled time. However, by industry standard, a train may arrive up to 5 minutes and 59 seconds after its scheduled time and still be considered on-time.

SEPTA has established an annual goal of 91% for Regional Rail On-Time Performance. How well are they doing? Is it even a meaningful measure?

In order to be able to answer such questions, we will try to design features that may help us **predict** delays of a train on a given **stop** at a given **date**.

### Reading the data

The SEPTA dataset consists of 2 CSV files that we will consider in this Starter Kit. The first file, `otp.csv`, provides OTP data, that is, information about trains and the times at which they arrive at the different stations on their lines.

In [ ]:
df_otp = sp.load_otp_data()
df_otp.head(10)

The different attributes in this file OTP dataset are as follows:
* `train_id`: the id of the train
* `direction`: the train direction; its values are 'N' for Northbound and 'S' for Southbound
* `origin`: the station before *next_station*
* `next_station`: the next station stop at *timeStamp*
* `date`: the day
* `status`: the current train delay; its values are 'On Time' or the delay, i.e. the amount of time above the 5min59s limit until which a train is considered to be on time (e.g. '1 min', '5 min', '10 min'); a value of 999 indicates a suspended train
* `timeStamp`: the timestamp at which the train will arrive at `next_station`

The second file, `trainView.csv`, provides train tracking information.

In [ ]:
df_train_view = sp.load_train_view_data()
df_train_view.head(3)

Its most important attributes are:
* `lon`: the current GPS longitude of the train
* `lat`: the current GPS latitude of the train
* `timeStamp0`: the earliest timestamp at the above GPS coordinates
* `timeStamp1`: the latest timeStamp at the above GPS coordinates
* `seconds`: the time difference (in seconds) between `timeStamp1` and `timeStamp0`

We can already observe in the previews of these two datasets format differences at the level of the `train_id` and `status` columns. We will address these in the next section.

## Data preprocessing


### Removing invalid data instances

A manual inspection of the dataset reveals that the `train_id` column contains 1 negative value and many values containing letters and punctuation characters. Let's filter out the rows for which the `train_id` is not a number.

In [ ]:
df_otp = sp.remove_invalid_data_instances(df=df_otp)

### Understanding train id

Train id may be an identifier for a specific run between a departure and a destination station at a given time of the day. To confirm that this is the case in this dataset, we will count how many times a _train_id_ passes through a given station on a given day:

In [ ]:
train_run = sp.get_train_run(df=df_otp)
print("In %0.2f%% of the cases, a train_id passes only once through a station on a given day." %
        (train_run.single_pass.sum() / float(len(train_run)) * 100))

We will exclude from the dataset the remaining 0.2% of cases, since they correspond to exceptional situations where a _train_id_ was registered more than once in a day.
For consistency, we only keep in the trainView dataset those rows for which the (`train_id`, `next_station`, `date`) tuples also occur in the OTP dataset.

For the remainder of the notebook, we will refer to _train_id_ as an identifier for a train journey between a given  _origin_ and a given _destination_ at a given _time_ of the day. A _train_run_ will be a _train_id_ on a specific date.

In [ ]:
df_otp, df_train_view = sp.exclude_remaining_cases(otp=df_otp, train_view=df_train_view)

### Turning statuses into workable delays

The current string-based format of the `status` column in the OTP dataset is not suitable for performing calculations on the delays. We will transform its values into integers (in minutes) and rename the column to `delay`.

In [ ]:
df_otp = sp.calculate_delays_for_otp(df=df_otp)

We will also exclude `None` values present in the `status` column of the df_train_view dataset, and, as for the OTP dataset, convert the values of that column to integers and rename the column to `delay`.

In [ ]:
df_train_view = sp.calculate_delays_for_train_view(df=df_train_view)

### Removing suspended trains

Since a status of `999` represented a suspended train, we exclude those trains from the datasets.
In addition, a delay of 1440 minutes corresponds to one day delay, meaning that the train was most likely canceled. We exclude the trains with those delays from the datasets as well.

In [ ]:
df_otp, df_train_view = sp.remove_suspended_trains(df_otp=df_otp, df_train_view=df_train_view)

## Data overview

Let's check some basic characteristics of the (cleaned OTP) dataset.

In [ ]:
sp.get_otp_overview(df=df_otp)

As we can see, the statement that 91% of all trains are on time (definition: less than 6 minutes delay) is dubious.

> RVEB: the delay is calculated as the time in minutes in the status column, but it is mentioned above that this status is the time above 5m59s of delay. I think the former is correct, but the latter explanation is not.

## Feature engineering

We are now ready to investigate several features to demonstrate the workflow and some hidden obstacles while calculating features. You might come up with other features as well, so don't hesitate to try them out.


### Day of the week

Based on our collective experience from using public transport, we can expect that delays are more likely to happen at specific moments in a day (e.g. rush hours) and at specific days in a week (e.g. working days). In order to be able to easily verify these assumptions later on, we first extract for each entry in the OTP dataset the following features:
- the hour
- the name of the day
- the type of day, which we define as having 3 possible values: weekday, Saturday and Sunday
- and a boolean indicating whether that day is a weekday or a weekend day

Let's start by looking at the impact of day of the week on delay.

In [ ]:
vis.plot_train_delays(df=df_otp)

As we can see, the difference in delays in weekdays (Mon-Fri) is only minimal. We will thus categorize the day of the week into weekdays, saturdays and sundays.

In [ ]:
df_otp = sp.get_type_of_days(df=df_otp)

### Month of the year

Seasonal effects, expressed here as the month of the year, can have an impact on train performance. As an example consider the summer holidays one can expect less passenger traffic, albeit also reduced staff, which might affect how on-time trains are. 
Let's start by looking at delay as a function of month of the year.

In [ ]:
vis.plot_train_delays(df=df_otp, type='month')

Data only runs from March to November 2016. Given the large variability observed between months, we will keep this feature, rather than splitting the months into the four seasons.

### Rush hours

During rush hours there's an intense increase in the number of passengers and, often, of the number of trains. It is also a period where it is most critical for trains to be on-time. 
To identify rush hour periods, we will start by extracting some time features from the dataset: the hour and whether it is a weekend day or not.

In [ ]:
df_otp = sp.extract_time_features(df=df_otp)
df_otp[['train_id', 'origin', 'date', 'hour', 'isWeekend']].sample(5)

Let's now use these features to count the number of distinct trains in the rail system per hour during weekdays, and hence have an idea of the train density per hour.
We define a rush hour as one for which the number of trains per hour and across all hours and weekdays is above the 75th percentile. 

Let's visualize the train density per hour on weekdays with that threshold.

In [ ]:
vis.plot_train_density(df=df_otp)

We will now define rush hour as all times on weekdays that fall in the hours above the red line.

> RVEB: which red line?

In [ ]:
df_otp = sp.define_rush_hour(df=df_otp, percentile=75)
df_otp[['train_id', 'origin', 'date', 'hour', 'dayOfTheWeek', 'isRushHour']].sample(5).query("isRushHour==True").head()

### Rank the different stops in a train run

The fact that a `train_id` passes only once through a station on a given day allows us to assume that a `train_id` 
is defined for a specific trip (origin-destination) at a given time in the day. So the combination between `train_id` and `date` will give us the train run.  

In [ ]:
df_otp = sp.calculate_rank_stop(df=df_otp)

We can now look at the delay as a function of the stop number.

In [ ]:
vis.plot_delays_per_stop_number(df=df_otp)

We can see that the delay is a function of the train stop number, confirming the idea that trains accumulate delays along the run, without being able of (fully) compensating for them.

### Distance between stations

We will now use the coordinates information to derive the distance between stations, the total distance per train run and the cumulative distance for each run. Since there are multiple coordinate points for each stop (train position might be registered at different time points, e.g. when entering the station, when docking, when departing, etc.), we will take the average latitude and longitude for each station.

In [ ]:
uni_coords, uni_runs, df_distance, df_coords = sp.calculate_distance_between_stations(df_otp=df_otp, df_train_view=df_train_view)
df_coords[['train_id', 'date', 'rk_stop', 'next_station', 'upcoming_station', 'distance']].sample(5)

We will now visualize an example of a _train__run_. We will plot the locations of all the train stations (in gray) with the stations of the example _train__run_ colored by the order they occur.

In [ ]:
vis.plot_train_run(df_train_view=df_train_view, uni_coords=uni_coords, uni_runs=uni_runs, df_distance=df_distance, df_coords=df_coords)

If we plot the distribution of distances we can see that that most inter-station distances are shorter than 10km.

In [ ]:
vis.plot_distance_distribution(df=df_coords)

### Cumulative distance along a _train__run_

This feature calculates how far a train has traveled along a given _train__run_. It is likely correlated with the rank of a given station on a given _train__run_. Nevertheless, it might convey different information and be useful, depending on the specific questions we will try to address, when applying this feature engineering analysis to a machine learning problem. 

In the table below we see the information regarding distance and cumulative distance for an example train run.

In [ ]:
df_coords, random_train_run = sp.calculate_cumulative_distance_along_train_run(df=df_coords, uni_runs=uni_runs)
random_train_run

In [ ]:
vis.delay_reg_plot(df=df_coords, uni_runs=uni_runs)

Interestingly, only cumulative distance appears to show a correlation with delay, suggesting that delay is not caused by the traveled distance between stations (and rather by delays incurred at stations themselves). Naturally, the longer the train run, the longer will be the delay incurred (as we have already shown with stop rank).

### Total distance of a _train__run_

The total distance traveled by the train is probably closely linked to the total delay or the delay at the last stop of a _train__run_. 
Here we will look at train run total distance and delay.

In [ ]:
vis.delay_jointplot(df=df_coords, uni_runs=uni_runs)

In [ ]:
vis.total_train_run_distance_reg_plot(df=df_coords)

In [ ]:
vis.total_train_run_distance_reg_plot(df=df_coords)

We can see that train run distance influences both the total delay throughout the run, as well as the delay at the last stop.

In [ ]:
df_otp = sp.add_cum_delay(df=df_otp)
df_otp = sp.add_distance(df_otp=df_otp, df_coords=df_coords)

### Northbound vs Southbound travels

This feature identifies stations which are typically problematic, in the sense that there are always delays. We can do this by checking their average delay.

We will first compare the delays of trains operating in one or the other direction.

In [ ]:
vis.plot_northbound_southbound_dely_per_station(df=df_otp)

The asymmetry in the scatterplot (even if not that striking) suggests that delay is dependent on direction (as could be expected, since the distance from origin and stop number will be different according to the direction), so we will label long-duration stations per direction.

### Stations with big delay

Some stations might be more prone to delays. This might be caused e.g. by many tracks converging to a smaller number of platforms of the station or due to long-term works on a station. 
We will, thus, rank the average delay for each station and label the top and bottom ones.

In [ ]:
df_otp

In [ ]:
vis.plot_stations_with_long_delays(df=df_otp)

We can label those stations with an average delay above a certain threshold. In this case, we will use the 90th percentile as a threshold.

In [ ]:
vis.plot_average_delay(df=df_otp)

This figure shows the distribution of average station delays. The red line indicates the threshold (90th percentile) that was used to label stations with considerably long delays.

In [ ]:
sp.print_delays_overview(df=df_otp)
df_otp = sp.label_long_delay_stations(df=df_otp)

### Delay over the last 7 days

Sometimes, uncontrollable factors can cause delays in trains (e.g. works on the train tracks, long duration strikes, etc.). We can calculate what was the delay over the last 7 days of each date to add that as a feature to predict delays.

In [ ]:
df_otp, last_week_delay = sp.calculate_longest_delays_over_past_week(df=df_otp)
print('10 cases with the longest delay over the past 7 days')
last_week_delay.sort_values('last_week_delay', ascending=False).head(10)

### Track changes

Track changes can be cause of delays. To address that, we will compare delays in cases when there was a track change 
with cases when there was no track changes.

In [ ]:
df_track_changes = sp.get_track_changes(df=df_train_view)

In [ ]:
vis.plot_track_changes(df=df_track_changes)

As we can see, delays are higher when there were track changes. A feature we can add to our model is the track change frequency per station per day, defined as:
$$\mathit{Track\space change\space frequency}_{d} = \frac{\sum_{tc\_tr}}{\sum_{tr}}$$
where $d$ is a given date, $tc\_tr$ is a train arrival with a track change and $tr$ is a train.

In [ ]:
track_change_frequency_date = sp.calculate_track_changes_frequency(df=df_track_changes)
track_change_frequency = sp.calculate_aggregated_track_changes(df=track_change_frequency_date)

We can compare how delay changes as a function of track change frequency:

In [ ]:
vis.plot_track_change_frequency_with_delays(df_track_change_frequency=track_change_frequency_date, df_otp=df_otp)

We can see that there's an upward trend (confirmed by the significant, positive correlation), suggesting that stations with higher delays incur more track changes.

In this plot we see the 10 stations with the highest track change frequency.

In [ ]:
# get unique combinations of station/coordinates and of train_id/dates
uni_coords = df_train_view[['next_station', 'lon', 'lat']].drop_duplicates()
uni_runs = df_otp[['train_id', 'date']].drop_duplicates()

In [ ]:
vis.plot_highest_track_change_frequency(track_change_frequency=track_change_frequency)

And the bottom 10

In [ ]:
vis.plot_lowest_track_change_frequency(track_change_frequency)

In [ ]:
# add track change frequency as a feature
df_otp = df_otp.merge(track_change_frequency, on='next_station')

## Final dataset preparation

Now we can create a dataset with the calculated features that is ready to be used in the modeling section.

In [ ]:
df_output = df_otp[['train_id', 'date', 'origin', 'next_station', 'timeStamp', 'delay', 'direction', 'hour',
                    'isWeekend', 'isRushHour', 'rk_stop', 'cum_delay', 'distance', 'cum_distance',
                    'long_delay_station', 'last_week_delay', 'track_change_frequency']]

df_output.head()

## Conclusion

In this Starter Kit we demonstrated the workflow on how to develop some features that might be valuable in this specific question: can we predict train delays at a given station and on a given day. We demonstrated the need of data preparation steps before calculating features as it might simplify the work later. In addition, for each derived feature, we show the impact it has on train delay.

## Additional information

This Starter Kit was developed in the context of the EluciDATA project [http://www.elucidata.be](http://www.elucidata.be). For more information, please contact [info@elucidata.be](info@elucidata.be).

Interested in more information? Visit our Master Course on Data Science ["The Art of Feature Engineering"](https://elucidatalab.be/mastercourse-data-innovation-beyond-hype#TheArtOfFeatureEngineering).


©, 2022, Sirris